In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine

def extract()-> dict:
    """ This API extracts data from
    http://universities.hipolabs.com
    """
    API_URL = "http://universities.hipolabs.com/search?country=United+States"
    data = requests.get(API_URL).json()
    return data

## Data Transformation

In [2]:
def transform(data:dict) -> pd.DataFrame:
    """ Transforms the dataset into desired structure"""
    df = pd.DataFrame(data)
    print(f"Total Number of universities from API {len(data)}")
    df['domains'] = [','.join(map(str, l)) for l in df['domains']]
    df['web_pages'] = [','.join(map(str, l)) for l in df['web_pages']]
    df = df.reset_index(drop=True)
    return df[["domains","country","web_pages","name"]]

def load(df:pd.DataFrame)-> None:
    """ Loads data into a sqllite database"""
    disk_engine = create_engine('sqlite:///my_lite_store.db')
    df.to_sql('cal_uni', disk_engine, if_exists='replace')

In [3]:
data = extract()
df = transform(data)
load(df)

Total Number of universities from API 4535


In [4]:
df.head()

,domains,country,web_pages,name
0,marywood.edu,United States,http://www.marywood.edu,Marywood University
1,lindenwood.edu,United States,http://www.lindenwood.edu/,Lindenwood University
2,sullivan.edu,United States,https://sullivan.edu/,Sullivan University
3,fscj.edu,United States,https://www.fscj.edu/,Florida State College at Jacksonville
4,xavier.edu,United States,https://www.xavier.edu/,Xavier University


## Another Method

In [5]:
from sqlite3 import connect
connection = connect('Database.db')
curr = connection.cursor()
curr.execute('CREATE TABLE IF NOT EXISTS University_data (domain TEXT, country TEXT, Webpade TEXT, name TEXT)')
# write the dataframe into the database table  
df.to_sql('University_data', connection, if_exists='replace')
curr.execute('''SELECT * FROM University_data''')
curr.execute('SELECT * FROM University_data')
data = curr.fetchmany(100)

In [6]:
data

[(0,
  'marywood.edu',
  'United States',
  'http://www.marywood.edu',
  'Marywood University'),
 (1,
  'lindenwood.edu',
  'United States',
  'http://www.lindenwood.edu/',
  'Lindenwood University'),
 (2,
  'sullivan.edu',
  'United States',
  'https://sullivan.edu/',
  'Sullivan University'),
 (3,
  'fscj.edu',
  'United States',
  'https://www.fscj.edu/',
  'Florida State College at Jacksonville'),
 (4,
  'xavier.edu',
  'United States',
  'https://www.xavier.edu/',
  'Xavier University'),
 (5,
  'tusculum.edu',
  'United States',
  'https://home.tusculum.edu/',
  'Tusculum College'),
 (6,
  'cst.edu',
  'United States',
  'https://cst.edu/',
  'Claremont School of Theology'),
 (7,
  'columbiasc.edu',
  'United States',
  'https://www.columbiasc.edu/',
  'Columbia College (SC)'),
 (8,
  'clpccd.edu',
  'United States',
  'http://www.clpccd.edu/',
  'Chabot-Las Positas Community College District'),
 (9,
  'keller.edu',
  'United States',
  'https://www.keller.edu/',
  'Keller Graduat